## Can trading using RSI beat buying and holding a stock?

### Background: 
The Relative Strength Index (RSI) is a momentum indicator that describes the current price relative to average high and low prices over a previous trading period. This indicator estimates overbought or oversold status and helps spot trend reversals, price pullbacks, and the emergence of bullish or bearish markets.

The indicator was originally developed by J. Welles Wilder Jr. and introduced in his seminal 1978 book, “New Concepts in Technical Trading Systems.”

An asset is usually considered overbought when the RSI is above 70% and oversold when it is below 30%.

Here is the process of calculating RSI:

1. For 14 periods, calculate the difference in price from the current period and the previous period;
2. For each period, record each positive change in price as a gain and each negative change as a loss;
3. On the 14th period, calculate the arithmetic mean of the gains and losses for the entire 14 day period (e.g. gains / 14 & losses / 14);
4. Use these values to calculate the RS
5. Use the RS value to calculate the RSI
6. For each proceeding period, use only the previous RSI value to calculate the next average value by multiplying by our lookback period – 1 (e.g. 13 for a lookback of 14).
7. Add the value obtained in step 6 to the current day’s value (do this for both the gains and losses)

<img src="rsi.jpeg" width=300 height=300 />

### Resource:

- https://www.alpharithms.com/relative-strength-index-rsi-in-python-470209
- https://www.tradingview.com/support/solutions/43000502338-relative-strength-index-rsi/
- https://en.wikipedia.org/wiki/Relative_strength_index

### Import Libraries and Finance Data 

In [1]:
# import basic libraries
import pandas as pd
import pandas_ta as ta
import numpy as np
import altair as alt
from IPython.display import display_html
from itertools import chain,cycle

# adjust the width of jupyter notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

#three ways to import finance data, we will mainly use yfinance 
#https://pypi.org/project/yfinance    OR  https://www.alpharithms.com/python-financial-data-491110/

# import quandl
# df = quandl.get('WIKI/NVDA')

# import pandas_datareader as pdr
# #request API call from https://data.nasdaq.com/sign-up
# df = pdr.get_data_quandl("NVDA", api_key="vjsci8v8EfV-BpoQmxbR")

import yfinance as yf

In [2]:
## Function to show dataframes side by side
## https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
def display(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [3]:
# Get finance data, valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
Ticker="tsla"
df = yf.Ticker(Ticker).history(period='3y')
df=df.reset_index()
df['Date']=pd.to_datetime(df['Date'])
df=df[['Date','Open','High','Low','Close']]
df['Close']=round(df['Close'],2)

In [4]:
df.head()

,Date,Open,High,Low,Close
0,2019-03-26,52.888000,54.051998,52.886002,53.55
1,2019-03-27,53.750000,55.074001,53.636002,54.97
2,2019-03-28,55.431999,56.066002,55.020000,55.72
3,2019-03-29,55.740002,56.032001,54.900002,55.97
4,2019-04-01,56.523998,57.840000,56.256001,57.84


### Calculating RSI Value

My attempt on calculating the RSI value using simple average. But a typical RSI value is calculating using Wilder’s original smoothing method
<img src="wsm.jpeg" width=300 height=300 />

In [5]:
#using the ta library, RSI values are calculated using daily close price. The default RSI period is 14 days.
df['RSI']=round(ta.rsi(df['Close'], timeperiod=14),2)

#Dropping the first 14 days without 14 days and reset index
df.drop(range(0,14),inplace=True)
df.reset_index(drop=True,inplace=True)

# Value is considered "overbought" when it exceeds 70 and "oversold" when indictator is below 30
df['Overbought']=70
df['Oversold']=30

In [6]:
# uncomment if you want to see how RSI is calculated using Simple Averages

# df['Difference'] = df['Close'].diff()
# df['Difference'].fillna(0,inplace=True)
# df['PercentChange'] = round(df['Difference']/ df['Close']*100,2)
# df.loc[df['Difference']>0, 'Gain'] = df['Difference']
# df.loc[df['Difference']<0, 'Loss'] = -df['Difference']
# df['Gain'].fillna(0,inplace=True)
# df['Loss'].fillna(0,inplace=True)
# for i in range(0,df.shape[0]-14):
#     sumGain=0
#     sumLoss=0
#     for index, row in df[1+i:14+i].iterrows():
#         sumGain=sumGain+row['Gain']
#         sumLoss=sumLoss+row['Loss']
#     df.loc[df.index==(14+i),'AvgGain']=round(sumGain/8,2)
#     df.loc[df.index==(14+i),'SumGain']=sumGain    
#     df.loc[df.index==(14+i),'AvgLoss']=round(sumLoss/8,2)    
#     df.loc[df.index==(14+i),'SumLoss']=sumLoss    
# df['RS'] = round(df['AvgGain']/df['AvgLoss'],2)
# df['AvgGain'].fillna(0,inplace=True)
# df['AvgLoss'].fillna(0,inplace=True)
# df['RS'].fillna(0,inplace=True)
# df['RSI'] = round(100-(100/(1+df['RS'])),2)
# df=df.reset_index()
# df.drop(range(0,14),inplace=True)

In [7]:
#Prepare of Stock graph
Stock = alt.Chart(df).mark_line(point=alt.OverlayMarkDef(color="grey",size=15)).encode(
    x='Date:T',
    y='Close:Q',
    tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('Close:Q', format=',.2f')]
).properties(
    title='Stock Price',
    width=800,
    height=300
)


In [8]:
#Plotting RSI chart with Oversold and Overbought lines
RSI = alt.Chart(df).mark_line().encode(
    x='Date:T',
    y='RSI:Q',
    tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('Close:Q', format=',.2f'),
             alt.Tooltip('RSI:Q', format=',.2f')]
).properties(
    title='RSI',
    width=800,
    height=300)

Oversold = alt.Chart(df).mark_line(color= 'red').encode(
    x='Date:T',
    y='Oversold:Q'
)

Overbought = alt.Chart(df).mark_line(color= 'green').encode(
    x='Date:T',
    y='Overbought:Q'
)

RSI_Chart=RSI+Oversold+Overbought
Stock & RSI_Chart

alt.VConcatChart(...)

### Setting up RSI Strategy 

In [9]:
print('Current Start Date :',df.iloc[0]['Date'])

Current Start Date : 2019-04-15 00:00:00


In [10]:
# Set up your initial investment and time below
df = df.loc[df['Date']>='2019-04-15']
df.reset_index(drop=True,inplace=True)
df.head()

,Date,Open,High,Low,Close,RSI,Overbought,Oversold
0,2019-04-15,53.726002,53.776001,51.726002,53.28,43.23,70,30
1,2019-04-16,53.150002,55.000000,52.944000,54.67,51.10,70,30
2,2019-04-17,54.950001,54.958000,53.708000,54.25,48.89,70,30
3,2019-04-18,54.245998,54.967999,53.950001,54.65,51.06,70,30
4,2019-04-22,53.799999,53.936001,52.495998,52.55,41.19,70,30


In [11]:
#initial values
startAmount=1000
startPrice=df.iloc[0]['Close']
shares=round(startAmount/startPrice,3)

In [12]:
# choose your sell and buy RSI threshold, you would sell when RSI hits the sell threhold and rebuy when RSI drops below buy thresold
RSI_Sell=70
RSI_Buy=60

In [13]:
#RSI Dateframe with Initialize Info
df_rsi = pd.DataFrame(columns = ['Date','Price','Shares', 'Amount','RSI','Buy/Sell','Profit/Loss','Profit/Loss %'])
df_rsi = df_rsi.append({'Date' : df.iloc[0]['Date'], 
              'Price' : startPrice,
              'Shares' : shares, 
              'Amount' : startAmount,
              'RSI': df.iloc[0]['RSI'],
              'Buy/Sell': 'Buy',
              'Profit/Loss':0,
              'Profit/Loss %':0},ignore_index=True)

df_longterm = df_rsi

print('Start Price:', '${:,.2f}'.format(startPrice))
print('Start Shares: ', shares)
print('RSI Sell Signal: >=', RSI_Sell)
print('RSI Buy Signal: <', RSI_Buy)
df_rsi.head()

Start Price: $53.28
Start Shares:  18.769
RSI Sell Signal: >= 70
RSI Buy Signal: < 60


,Date,Price,Shares,Amount,RSI,Buy/Sell,Profit/Loss,Profit/Loss %
0,2019-04-15,53.28,18.769,1000,43.23,Buy,0,0


In [14]:
## buy and sell stock based on RSI value
Sell_Flag=0
for index, row in df.iterrows():
    if index!=0 and row['RSI']>=RSI_Sell and Sell_Flag==0:
        Sell_Flag=1
        Sold_Amount=df_rsi.iloc[-1]['Shares']*row['Close']
        df_rsi = df_rsi.append({
              'Date' : row['Date'], 
              'Price' : row['Close'],
              'Shares' : 0, 
              'Amount' : Sold_Amount,
              'RSI': row['RSI'],
              'Buy/Sell': 'Sell',
              'Profit/Loss': round(Sold_Amount-df_rsi.iloc[-1]['Amount'],2),
              'Profit/Loss %': round((Sold_Amount-df_rsi.iloc[-1]['Amount'])/df_rsi.iloc[-1]['Amount']*100,2)                              
                            },ignore_index=True)
    
    elif index!=0 and row['RSI']<RSI_Buy and Sell_Flag==1:
        Sell_Flag=0
        df_rsi = df_rsi.append({
              'Date' : row['Date'], 
              'Price' : row['Close'],
              'Shares' : df_rsi.iloc[-1]['Amount']/row['Close'], 
              'Amount' : df_rsi.iloc[-1]['Amount'],
              'RSI': row['RSI'],
              'Buy/Sell': 'Buy',
              'Profit/Loss':0,
              'Profit/Loss %':0                              
                            },ignore_index=True)
        
        
# sell on the last day and calculate the final profit/loss        
if df_rsi.iloc[-1]['Buy/Sell']=='Buy':
    Sold_Amount=df.iloc[-1]['Close']*df_rsi.iloc[-1]['Shares']
    
    df_rsi = df_rsi.append({
              'Date' : df.iloc[-1]['Date'], 
              'Price' : df.iloc[-1]['Close'],
              'Shares' : 0, 
              'Amount' : Sold_Amount,
              'RSI': df.iloc[-1]['RSI'],
              'Buy/Sell': 'Sell',
              'Profit/Loss':Sold_Amount-df_rsi.iloc[-1]['Amount'],
              'Profit/Loss %':round((Sold_Amount-df_rsi.iloc[-1]['Amount'])/df_rsi.iloc[-1]['Amount']*100,2)                              
                            },ignore_index=True)
    
# add buy and sell threshold
df_rsi['Sell'] = RSI_Sell
df_rsi['Buy'] = RSI_Buy

In [15]:
df_rsi

,Date,Price,Shares,Amount,RSI,Buy/Sell,Profit/Loss,Profit/Loss %,Sell,Buy
0,2019-04-15,53.28,18.769000,1000,43.23,Buy,0,0,70,60
1,2019-07-15,50.70,0.000000,951.588,72.41,Sell,-48.41,-4.84,70,60
2,2019-07-25,45.76,20.795199,951.588,43.54,Buy,0,0,70,60
3,2019-10-24,59.94,0.000000,1246.46,78.58,Sell,294.88,30.99,70,60
4,2019-11-22,66.61,18.712869,1246.46,56.68,Buy,0,0,70,60
5,2019-12-16,76.30,0.000000,1427.79,76.12,Sell,181.33,14.55,70,60
6,2020-02-25,159.98,8.924815,1427.79,58.40,Buy,0,0,70,60
7,2020-06-08,189.98,0.000000,1695.54,71.67,Sell,267.74,18.75,70,60
8,2020-06-24,192.17,8.823107,1695.54,57.03,Buy,0,0,70,60
9,2020-07-01,223.93,0.000000,1975.76,71.27,Sell,280.22,16.53,70,60


In [16]:
Final_RSI = alt.Chart(df_rsi).mark_circle(color='Blue',size=90).encode(
    x='Date:T',
    y='RSI:Q',
    color='Buy/Sell',
        tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('RSI:Q', format=',.2f')]
).properties(
    width=800,
    height=300
)

Sell_Line = alt.Chart(df_rsi).mark_line(strokeDash=[5,5],color= 'red').encode(
    x='Date:T',
    y='Sell:Q',
    tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('RSI:Q', format=',.2f')]
)

Buy_Line = alt.Chart(df_rsi).mark_line(strokeDash=[5,5], color= 'green').encode(
    x='Date:T',
    y='Buy:Q',
    tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('RSI:Q', format=',.2f')]
)

#RSI+Final_RSI+Sell_Line+Buy_Line

In [17]:
df_longterm = df_rsi.iloc[[0, -1]]
df_longterm.reset_index(drop=True,inplace=True)
df_longterm.at[1, 'Amount'] = df_longterm.iloc[-1]['Price']*df_longterm.iloc[0]['Shares']
df_longterm.at[1, 'Buy/Sell'] = 'Sell'
df_longterm.at[1, 'Profit/Loss'] = df_longterm.iloc[-1]['Amount']-df_longterm.iloc[0]['Amount']
df_longterm.at[1, 'Profit/Loss %'] = round(df_longterm.at[1, 'Profit/Loss']/df_longterm.iloc[0]['Amount']*100,2)

In [18]:
Stock&(RSI+Final_RSI+Sell_Line+Buy_Line)

alt.VConcatChart(...)

In [19]:
display(df_rsi,df_longterm, titles=['RSI Strategy ','Long Term Buy&Hold'])

,Date,Price,Shares,Amount,RSI,Buy/Sell,Profit/Loss,Profit/Loss %,Sell,Buy
0,2019-04-15,53.28,18.769000,1000,43.23,Buy,0,0,70,60
1,2019-07-15,50.70,0.000000,951.588,72.41,Sell,-48.41,-4.84,70,60
2,2019-07-25,45.76,20.795199,951.588,43.54,Buy,0,0,70,60
3,2019-10-24,59.94,0.000000,1246.46,78.58,Sell,294.88,30.99,70,60
4,2019-11-22,66.61,18.712869,1246.46,56.68,Buy,0,0,70,60
5,2019-12-16,76.30,0.000000,1427.79,76.12,Sell,181.33,14.55,70,60
6,2020-02-25,159.98,8.924815,1427.79,58.40,Buy,0,0,70,60
7,2020-06-08,189.98,0.000000,1695.54,71.67,Sell,267.74,18.75,70,60
8,2020-06-24,192.17,8.823107,1695.54,57.03,Buy,0,0,70,60
9,2020-07-01,223.93,0.000000,1975.76,71.27,Sell,280.22,16.53,70,60
